In [3]:
%%capture
import matplotlib
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta, datetime, time, date
from matplotlib import rc
from tqdm.notebook import tqdm
from cycler import cycler
tqdm().pandas()

fontsize = 12
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': fontsize})
rc('text', usetex=True)

color_list = ["#348ABD","#A60628","#7A68A6","#467821","#CF4457","#188487","#E24A33"]
style = {
  "lines.linewidth": 2.0,
  "axes.edgecolor": "#bcbcbc",
  "patch.linewidth": 0.5,
  "legend.fancybox": True,
  "axes.prop_cycle": cycler('color', color_list),
  "axes.facecolor": "#ffffff",
  "axes.labelsize": "large",
  "axes.grid": True,
  "patch.edgecolor": "#eeeeee",
  "axes.titlesize": "x-large",
  "svg.fonttype": "path"}

matplotlib.rcParams.update(style)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)


data_path           = 'C:/Users/Stefa/Documents/Uni/Projektassistenz/Python/Data/'
output_path         = 'C:/Users/Stefa/Documents/Uni/Projektassistenz/Auswertung/'
fin_data_path       = 'C:/Users/Stefa/Documents/Uni/Projektassistenz/Financial Data/'   
extended_model_path = data_path+'Classification/DagoBERT/SCE_Loss_minw_25_lr_5e5_3ep_bs32_wd_1e2_a0_5_b3_NN1_w2v_topics/' 


def load_data(file):
    df = pd.read_csv(file, encoding='utf-8-sig')
    df.Date = pd.to_datetime(df.Date)
    df = df.set_index(['Ticker', 'Date'])
    df = df.rename(columns={'Prediction value': 'prediction_value'})
    return df


def get_zscores(x, idiosync_z_scores, rw=int(255/2)):
    idiosync = '_idiosync' if idiosync_z_scores else ''
    
    r = x.rolling(window=rw, closed='left', min_periods=int(0.9*rw))
    m = r.mean()
    s = r.std()
    z = (x-m)/s
    z.index.name = 'Date'    
    return z

Load Data

In [4]:
no_dupl = True                # Load news events without duplicates
return_period = 'mo_to_mc'    # mo_to_mo, mc_to_mc 
                              # mo_to_mc, mc_to_mo

z_cols = ['z_value_t_III', 'z_value_t_II', 'z_value_t_I', 'z_value_t', 'z_value_t_1']
r_cols = ['return_t_V', 'return_t_IV', 'return_t_III', 'return_t_II', 'return_t_I', 'return_t', 'return_t_1', 
          'return_t_2', 'return_t_3', 'return_t_4', 'return_t_5', 'return_t_6', 'return_t_7', 'return_t_8']
n_cols = []


# Load News Events Data
if no_dupl:
    news_event_df = pd.read_csv('./Data/news_event_df_no_dupl.csv', encoding='utf-8', index_col=0)
    news_event_df.Date = pd.to_datetime(news_event_df.Date)
    news_event_df = news_event_df.set_index(['Date', 'Ticker']) 
    news_event_df = news_event_df.drop(['prev_close_date'], axis=1)    
else:
    news_event_df = pd.read_csv('./Data/news_event_df.csv', encoding='utf-8')
    news_event_df.Timestamp_ET = pd.DatetimeIndex(news_event_df.Timestamp_ET)
    news_event_df.Date = pd.to_datetime(news_event_df.Date)
    news_event_df = news_event_df.set_index(['Timestamp_ET', 'Ticker'])


# Return data of all constituents (file reference: Stock Return Calculation.ipynb)     
asset_returns = pd.read_csv(data_path + f"return_{return_period}.csv", index_col=0)                 
asset_returns.index = pd.to_datetime(asset_returns.index)                                                        
asset_returns = asset_returns.replace([np.inf, -np.inf, 0.00000], np.nan)              
asset_returns = asset_returns[asset_returns.apply(lambda x: sum(x.isna()), axis=1) < 0.8*asset_returns.shape[1]] # Drop rows with more than 80% NaN values
return_data = asset_returns.loc[news_event_df.index.get_level_values(0)[0]-timedelta(days=1) : news_event_df.index.get_level_values(0)[-1]]

# Load Beta values (file reference: Volatility adjusted labels.ipynb)
beta = pd.read_csv(data_path+'beta_moc_df.csv')     
beta.Date = pd.to_datetime(beta.Date)
beta = beta.set_index('Date')
                                        
Ticker_inSP = pd.read_csv(data_path+'Ticker_inSP_2020.csv', index_col=0)
Ticker_inSP.index = pd.to_datetime(Ticker_inSP.index)
dates_daily_freq = pd.date_range(start=datetime(2002,1,1), end=datetime(2020,2,28), freq='D').to_frame().loc[:, []]
Ticker_inSP_adj  = pd.merge_asof(left=dates_daily_freq, right=Ticker_inSP, left_index=True, right_index=True, direction='backward')

sp500 = pd.read_csv(fin_data_path+'SPCOMP_1994_2022.csv', sep=';', index_col=0)
sp500.index = pd.to_datetime(sp500.index) 
sp500['TOT RETURN OPEN'] = (sp500['OPENING PRICE']/sp500['PRICE INDEX'])*sp500['TOT RETURN IND']
sp500_r = sp500.pct_change()  
sp500_r['TR OpenClose'] = (sp500['TOT RETURN IND']  - sp500['TOT RETURN OPEN'])/sp500['TOT RETURN OPEN']
sp500_r['TR CloseOpen'] = (sp500['TOT RETURN OPEN'] - sp500['TOT RETURN IND'].shift(1))/sp500['TOT RETURN IND'].shift(1)

rf_3M = pd.read_csv(data_path+'rf_3M.csv')
rf_3M.Date = pd.to_datetime(rf_3M.Date)
rf_3M = rf_3M.set_index('Date')
rf_3M = rf_3M.rename(columns={'Value': 'rf_rate'})
rf_3M = pd.merge_asof(left=sp500.loc[:, []], right=rf_3M, left_index=True, right_index=True, direction='backward')

Calculate idiosyncratic returns  
*r_idiosync = r_j - r_f - beta(j, t-1)* (r_SP500(t) - r_f)*

In [5]:
def capm_expected_returns(beta_i):
    if return_period   == 'mc_to_mc':
        expected_return = expected_returns.rf_rate + beta_i * (expected_returns['Total Return'] - expected_returns.rf_rate)
    elif return_period == 'mo_to_mo':
        expected_return = expected_returns.rf_rate + beta_i * (expected_returns['Total Return Open']  - expected_returns.rf_rate)
    elif return_period == 'mo_to_mc':
        expected_return = expected_returns.rf_rate + beta_i * (expected_returns['TR OpenClose'] - expected_returns.rf_rate)    
    elif return_period == 'mc_to_mo':
        expected_return = expected_returns.rf_rate + beta_i * (expected_returns['TR CloseOpen'] - expected_returns.rf_rate)                  
    return expected_return

if return_period   == 'mc_to_mc':
    expected_returns = pd.merge(left=beta.shift(periods=1), right=sp500_r['Total Return'], left_index=True, right_index=True)
elif return_period == 'mo_to_mo':
    expected_returns = pd.merge(left=beta.shift(periods=1), right=sp500_r['Total Return Open'], left_index=True, right_index=True)
elif return_period == 'mo_to_mc':   
    expected_returns = pd.merge(left=beta.shift(periods=1), right=sp500_r['TR OpenClose'],  left_index=True, right_index=True)
elif return_period == 'mc_to_mo':
    expected_returns = pd.merge(left=beta.shift(periods=1), right=sp500_r['TR CloseOpen'],  left_index=True, right_index=True)

expected_returns = pd.merge(left=expected_returns, right=rf_3M,  left_index=True, right_index=True)  
expected_returns = expected_returns.loc[:,'MLM':'MU'].apply(capm_expected_returns)
abnormal_returns = asset_returns.loc[expected_returns.index[0]:expected_returns.index[-1]] - expected_returns
abnormal_returns = abnormal_returns.loc[news_event_df.index.get_level_values(0)[0]-timedelta(days=1) : news_event_df.index.get_level_values(0)[-1]]

#abnormal_returns.to_csv(data_path+f"return_{return_period}_idiosync.csv", encoding='utf-8')    

Match z-scores with news events and asset returns

In [7]:
use_abnormal_returns  = True
idiosync_z_scores     = True

def match_events_with_returns(news_event_df, use_abnormal_returns, no_dupl, news_window=17.5, order_time='moo'): 
    global Z, R, N, ZR, ZRN, dataset
    """
    If no_dupl = True we have no exact timestamps of the news arrival available. In this case, the index column Date is the
    date at market open. Thus, overnight news ar linked with the next market open date. 
    
    For daytime news with order_time = 'moc': no_dupl = False
    """    
    z_values = pd.read_csv(data_path+f'z_values_mo_to_mc_127d_rw{idiosync}.csv', encoding='utf-8')   
    z_values.Date = pd.to_datetime(z_values.Date)                                   
    z_values = z_values.set_index('Date') 

    if use_abnormal_returns:
        return_df = abnormal_returns
    else:
        return_df = return_data
    
    news_window_h   = int(news_window)
    news_window_min = int((news_window % 1)*60)
    
    dates = list(z_values.loc[datetime(2002,1,2):datetime(2020,1,31)].index)
    news_dates = set(news_event_df.index.get_level_values(0).unique())
    z_values_df = pd.merge(left=pd.DataFrame(index=dates), right=z_values, left_index=True, right_index=True, how='left')

    z_values  = z_values.loc[dates]
    z_values  = z_values.stack(dropna=False)
    return_df = pd.merge(left=pd.DataFrame(index=dates), right=return_df, left_index=True, right_index=True, how='left')
    
    for i, date in tqdm(enumerate(dates[:-8])):
        if i >= 5:            
            ticker_in_sp500_t = set(Ticker_inSP_adj.loc[dates[i-1], :].dropna().values)                
            Z = z_values.loc[dates[i-1], slice(None)].to_frame(name='z_value').reset_index(level=0)
            Z = Z.loc[list(set(Z.index).intersection(ticker_in_sp500_t))]          # Select only assets that are in the S&P500      
            Z.Date = date
            R = return_df.loc[dates[i-5:i+8+1], Z.index].T
            R.columns = r_cols
            
            if date in news_dates:
                if no_dupl:
                    if order_time == 'moo':
                        N = news_event_df.loc[(dates[i], slice(None)), :]
                    else:
                        print("Use oder_time='moo' or set no_dupl=False")
                        break                    
                else:                    
                    if order_time == 'moo':    # Market open order
                        rw_end   = datetime.combine(dates[i], time(9,30))
                        rw_start = rw_end - timedelta(hours=news_window_h, minutes=news_window_min)
                    elif order_time == 'moc':  # Market close order
                        rw_end   = datetime.combine(dates[i], time(16,0))
                        rw_start = rw_end - timedelta(hours=news_window_h, minutes=news_window_min)                
                        
                    N = news_event_df.loc[((rw_start <= news_event_df.index.get_level_values('Timestamp_ET')) & (news_event_df.index.get_level_values('Timestamp_ET') <= rw_end), slice(None)), :]            
            else:
                N = pd.DataFrame(columns=news_event_df.columns)
                              
            ZRN = Z.join([R, N.reset_index(level=0, drop=True)], how='left').reset_index()
            
            if i == 5:
                dataset = ZRN  
            else:
                dataset = pd.concat([dataset, ZRN], axis=0)
                      
    dataset = dataset.rename(columns={'index':'Ticker'})
    return dataset


abn           = '_abn' if use_abnormal_returns else '' 
idiosync      = '_idiosync' if idiosync_z_scores else ''
no_duplicates = '_no_dupl'  if no_dupl else ''

if return_period == 'mc_to_mc':
    new_df = match_events_with_returns(news_event_df, use_abnormal_returns, no_dupl, news_window=6.5,  order_time='moc')    # order_time: 'moc' or 'moo'
    new_df.to_csv(f'./Data/event_data{abn}_returns_mc_to_mc_6_5h_z_val{idiosync}{no_duplicates}_'+str(2002)+'-'+str(2021)+'.csv', encoding='utf-8-sig', index=False)  
    
if return_period == 'mo_to_mo':
    new_df = match_events_with_returns(news_event_df, use_abnormal_returns, no_dupl, news_window=17.5, order_time='moo')  
    new_df.to_csv(f'./Data/event_data{abn}_returns_mo_to_mo_17_5h_z_val{idiosync}{no_duplicates}_'+str(2002)+'-'+str(2021)+'.csv', encoding='utf-8-sig', index=False)      
    
if return_period == 'mo_to_mc':
    new_df = match_events_with_returns(news_event_df, use_abnormal_returns, no_dupl, news_window=17.5, order_time='moo')  
    new_df.to_csv(f'./Data/event_data{abn}_returns_mo_to_mc_17_5h_z_val{idiosync}{no_duplicates}_'+str(2002)+'-'+str(2021)+'_v1.csv', encoding='utf-8-sig', index=False)   
    
if return_period == 'mc_to_mo':
    new_df = match_events_with_returns(news_event_df, use_abnormal_returns, no_dupl, news_window=17.5, order_time='moo')  
    new_df.to_csv(f'./Data/event_data{abn}_returns_mc_to_mo_17_5h_z_val{idiosync}{no_duplicates}_'+str(2002)+'-'+str(2021)+'.csv', encoding='utf-8-sig', index=False)

0it [00:00, ?it/s]

Combine close-to-open and open-to-close returns in one file

In [3]:
beta_o = beta.copy()
beta_c = beta.copy()
beta_o.index = beta_o.index + timedelta(hours=9, minutes=30)
beta_c.index = beta_c.index + timedelta(hours=16, minutes=0)
beta_oc = pd.concat([beta_o, beta_c]).sort_index()

sp500_close_open = sp500_r['TR CloseOpen']
sp500_open_close = sp500_r['TR OpenClose']
sp500_close_open.index = sp500_close_open.index + timedelta(hours=9, minutes=30)
sp500_open_close.index = sp500_open_close.index + timedelta(hours=16, minutes=0)
sp500_returns = pd.concat([sp500_close_open, sp500_open_close], axis=0).sort_index()
sp500_returns.name = 'TR_SP500'

rf_3M_o = rf_3M.copy()
rf_3M_c = rf_3M.copy()
rf_3M_o.index = rf_3M_o.index + timedelta(hours=9, minutes=30)
rf_3M_c.index = rf_3M_c.index + timedelta(hours=16, minutes=0)
rf_3M_oc = pd.concat([rf_3M_o, rf_3M_c]).sort_index()

R_mo_to_mc = pd.read_csv(data_path + 'return_mo_to_mc.csv', index_col=0)          
R_mo_to_mc.index = pd.to_datetime(R_mo_to_mc.index)                                                        
R_mo_to_mc = R_mo_to_mc.replace([np.inf, -np.inf, 0.00000], np.nan)
R_mo_to_mc.insert(0, 'interval', 'mo_to_mc')
R_mo_to_mc.index = R_mo_to_mc.index + timedelta(hours=16)

R_mc_to_mo = pd.read_csv(data_path + 'return_mc_to_mo.csv', index_col=0)          
R_mc_to_mo.index = pd.to_datetime(R_mc_to_mo.index)                                                        
R_mc_to_mo = R_mc_to_mo.replace([np.inf, -np.inf, 0.00000], np.nan)              
R_mc_to_mo.insert(0, 'interval', 'mc_to_mo')
R_mc_to_mo.index = R_mc_to_mo.index + timedelta(hours=9, minutes=30)

asset_returns = pd.concat([R_mo_to_mc, R_mc_to_mo], axis=0).sort_index()
return_data = asset_returns.loc[news_event_df.index.get_level_values(0)[0]-timedelta(days=1) : news_event_df.index.get_level_values(0)[-1]]
#R_mc_to_mo = R_mc_to_mo[R_mc_to_mo.apply(lambda x: sum(x.isna()), axis=1) < 0.8*R_mc_to_mo.shape[1]] # Drop rows with more than 80% NaN values

In [4]:
def capm_expected_returns(beta_i):
    expected_return = expected_returns.rf_rate + beta_i * (expected_returns['TR_SP500'] - expected_returns.rf_rate)        
    return expected_return

expected_returns = pd.merge(left=beta_oc.shift(periods=1), right=sp500_returns, left_index=True, right_index=True)
expected_returns = pd.merge(left=expected_returns, right=rf_3M_oc, left_index=True, right_index=True)  

expected_returns = expected_returns.loc[:,'MLM':'MU'].apply(capm_expected_returns)
abnormal_returns = asset_returns.loc[expected_returns.index[0]:expected_returns.index[-1]] - expected_returns

In [10]:
def match_news_with_returns(news_event_df, use_abnormal_returns, news_window=17.5, order_time='moo'):    
    
    return_df = abnormal_returns if use_abnormal_returns else return_data
    
    news_window_h   = int(news_window)
    news_window_min = int((news_window % 1)*60)
    timestamps = list(return_data.index)
    dates = list(news_event_df.index.get_level_values(0).unique())
    print(len(dates))
    
    z_scores_oo = pd.read_csv(data_path+f'z_values_mo_to_mo_127d_rw{idiosync}.csv', encoding='utf-8')      
    z_scores_oo.Date = pd.to_datetime(z_scores_oo.Date)                                   
    z_scores_oo = z_scores_oo.set_index('Date')  
    z_scores_oo.index = z_scores_oo.index + timedelta(hours=9, minutes=30)
    z_scores_oo.insert(0, 'interval', 'mo_to_mo')

    z_scores_oc = pd.read_csv(data_path+f'z_values_mo_to_mc_127d_rw{idiosync}.csv', encoding='utf-8')      
    z_scores_oc.Date = pd.to_datetime(z_scores_oc.Date)                                   
    z_scores_oc = z_scores_oc.set_index('Date')  
    z_scores_oc.index = z_scores_oc.index + timedelta(hours=16, minutes=0)
    z_scores_oc.insert(0, 'interval', 'mo_to_mc')

    z_scores = pd.concat([z_scores_oo, z_scores_oc]).sort_index()
    z_scores = pd.merge(left=pd.DataFrame(index=timestamps), right=z_scores, left_index=True, right_index=True, how='left')

        
    for i, date in tqdm(enumerate(dates[:-8])):
        if i >= 5:
            if no_dupl:
                if order_time == 'moo':
                    df = news_event_df.loc[(dates[i], slice(None)), :]
                else:
                    print("Use oder_time='moo' or set no_dupl=False")
                    break                    
            else:   
                if order_time == 'moo': 
                    rw_end   = datetime.combine(date, time(9,30))
                    rw_start = rw_end - timedelta(hours=news_window_h, minutes=news_window_min)

                df = news_event_df.loc[((rw_start <= news_event_df.index.get_level_values('Timestamp_ET')) & 
                                         (news_event_df.index.get_level_values('Timestamp_ET') <= rw_end), slice(None)), 
                                         ['Date', 'Sentiment', 'freshness', 'topic_1', 'topic_2', 'topic_3', 'topic_4']]
            
            
            ticker = df.index.get_level_values('Ticker').unique()

            for j, return_j in enumerate(r_cols):
                for t in ['o', 'c']:
                    col = return_j + t
                    if t == 'o':           
                        r_j = return_df.loc[datetime.combine(dates[i-5+j], time(9,30)), ticker].rename(col)
                    elif t == 'c':
                        r_j = return_df.loc[datetime.combine(dates[i-5+j], time(16,0)), ticker].rename(col)

                    if ((j == 0) & (t == 'o')):
                        values = r_j
                    else:
                        values = pd.concat([values, r_j], axis=1)

                        
            for j, zval_j in enumerate(z_cols):
                for t in ['o', 'c']:
                    col = zval_j + t
                    if t == 'o':                     
                        z_j = z_scores.loc[datetime.combine(dates[i-3+j], time(9,30)),  ticker].rename(col)
                    elif t == 'c':
                        z_j = z_scores.loc[datetime.combine(dates[i-3+j], time(16,0)),  ticker].rename(col)

                    values = pd.concat([values, z_j], axis=1)
            
            if i == 5:
                new_df = pd.merge(df.reset_index(), values, on='Ticker', how = 'inner')
            else:
                new_df = pd.concat([new_df, pd.merge(df.reset_index(), values, on='Ticker', how = 'inner')])

    return new_df



use_abnormal_returns  = True
idiosync_z_scores     = True
abn = '_abn' if use_abnormal_returns else '' 
idiosync = '_idiosync' if idiosync_z_scores else ''
no_duplicates = '_no_dupl'  if no_dupl else ''

new_df = match_news_with_returns(news_event_df, use_abnormal_returns, news_window=17.5, order_time='moo')
new_df.to_csv(f'./Data/event_data{abn}_returns_oc_17_5h_z_val{idiosync}{no_duplicates}_'+str(2002)+'-'+str(2021)+'.csv', encoding='utf-8-sig', index=False)  

4517


0it [00:00, ?it/s]

Calculate Beta values

In [ ]:
r_w  = 252*2  # rolling window of 2 years
beta = pd.DataFrame(data=np.nan, index=return_data.index, columns=return_data.columns)


# Load return data
return_data = pd.read_csv(data_path + 'return_mo_to_mo.csv')
return_data.Date = pd.to_datetime(return_data.Date)
return_data = return_data.set_index('Date')

# S&P total return data
sp500 = pd.read_csv(data_path+'SP500_historical.csv')
sp500['sp500_return'] = sp500['Adj Close'].pct_change()
sp500.Date = pd.to_datetime(sp500.Date)
sp500 = sp500.set_index('Date')
sp500 = sp500[sp500.index >= pd.Timestamp(1990,1,2)]
sp500 = sp500[sp500.index <= pd.Timestamp(2020,11,20)]

return_data = pd.concat([return_data, sp500.sp500_return], axis=1, join='outer', sort=False)
return_data = return_data.replace([np.inf, -np.inf, 0.00000], np.nan)


for row in tqdm(range(r_w+1, len(return_data.index))):     
    dateStr = return_data.index[row]
    
    # Rolling window
    return_data_rw = return_data.iloc[(row-1)-r_w:(row-1), :]
    
    # Select all tickers that have at max. 40% NaN values in the rolling window  
    sel_ticker = return_data_rw.isna().sum() < 0.4*r_w  
    return_data_sel = return_data_rw[return_data.columns[sel_ticker]]

    # 3. Calculate covariances cov(r_i, r_m)
    cov = return_data_sel.cov() 

    # 4. Calculate var(r_m)
    var_r_m = np.var(return_data_rw.sp500_return.dropna())

    beta.loc[dateStr, :] = cov.sp500_return/var_r_m
        
beta = beta[r_w+1:]        
beta.to_csv(data_path+'beta_moc_df.csv', index=True)

Calculate z-scores

In [ ]:
idiosync_z_scores = True
idiosync = '_idiosync' if idiosync_z_scores else ''

if idiosync_z_scores:
    z_scores = get_zscores(abnormal_returns, idiosync_z_scores)    
else: 
    z_scores = get_zscores(asset_returns, idiosync_z_scores)    
    
z_scores.to_csv(data_path+f'z_values_mo_to_mo_127d_rw{idiosync}.csv', encoding='utf-8', index=True)

Calculate close-to-open return

In [46]:
tot_return_open  = pd.read_csv(data_path+'tot_return_open_index.csv', low_memory=False)
tot_return_close = pd.read_csv(data_path+'tot_return_close_index.csv', low_memory=False)
tot_return_open['Date']  = pd.to_datetime(tot_return_open['Date']) 
tot_return_close['Date'] = pd.to_datetime(tot_return_close['Date']) 
tot_return_open  = tot_return_open.set_index('Date')
tot_return_close = tot_return_close.set_index('Date')
return_close_to_open = (tot_return_open - tot_return_close.shift(periods=1))/tot_return_close.shift(periods=1)
return_close_to_open.to_csv(data_path+'return_mc_to_mo.csv', encoding='utf-8', index=True)

Calculate open-to-close return

In [ ]:
tot_return_open  = pd.read_csv(data_path+'tot_return_open_index.csv', low_memory=False)
tot_return_close = pd.read_csv(data_path+'tot_return_close_index.csv', low_memory=False)
tot_return_open['Date']  = pd.to_datetime(tot_return_open['Date']) 
tot_return_close['Date'] = pd.to_datetime(tot_return_close['Date']) 
tot_return_open  = tot_return_open.set_index('Date')
tot_return_close = tot_return_close.set_index('Date')

return_open_to_close = (tot_return_close-tot_return_open)/tot_return_open
return_open_to_close.to_csv(data_path+'return_mo_to_mc.csv', encoding='utf-8', index=True)

Creat Simplified News Events Dataframe  
*Source File: 'train_valid_data_inkl_pred_fresh_w2v_topics_2002-2021.csv'*

In [271]:
news_event_df = pd.read_csv(extended_model_path+'train_valid_data_inkl_pred_fresh_w2v_topics_'+str(2002)+'-'+str(2021)+'.csv', encoding='utf-8-sig', index_col=0)
news_event_df.Timestamp_ET = pd.DatetimeIndex(news_event_df.Timestamp_ET).tz_localize(None)
news_event_df.Date = pd.to_datetime(news_event_df.Date)
news_event_df = news_event_df.sort_values((['Timestamp_ET']),ascending=True)
news_event_df = news_event_df.set_index(['Timestamp_ET'])[:-1]
news_event_df = news_event_df.drop(['Sentiment'], axis=1)
news_event_df['Sentiment'] = news_event_df.positive-news_event_df.negative 

# Insert TradingDate column for news released during after trading hours on date t 
# to show the date t+1 of market opening on the next trading day
news_event_df['TradingDate'] = news_event_df.Date.copy()

cols = ['Date', 'TradingDate', 'Ticker', 'Sentiment', 'freshness', 'topic_1', 'topic_2', 'topic_3', 'topic_4']
news_event_df = news_event_df[cols]

trading_days   = np.array(list(return_data.index))
news_event_df  = news_event_df.loc[news_event_df.Date.isin(trading_days[:-1])]

def get_market_open_date(t):
    return trading_days[np.where(trading_days == t)[0][0] + 1]

market_open_date = news_event_df.loc[news_event_df.index.time > time(16,0), 'Date'].apply(get_market_open_date)
news_event_df.loc[news_event_df.index.time > time(16,0), 'TradingDate'] = market_open_date.copy()

news_event_df = news_event_df.reset_index()
news_event_df = news_event_df.set_index(['Timestamp_ET', 'Ticker'])

news_event_df.to_csv('./Data/news_event_df.csv', encoding='utf-8')

Remove duplicate overnight news (pre- and after midnight) from news_event_df

In [273]:
# News published between 4pm am 12am are merged into one document and news published between
# 12am an 9:30am are merged into a seperate document. Thus there can exist two seperate overnight 
# news documents of one company. This skript removes duplicates and calculates the mean of the sentiment 
# of two news articles.

# Load News Events Data
news_event_df = pd.read_csv('./Data/news_event_df.csv', encoding='utf-8')
news_event_df.Timestamp_ET = pd.DatetimeIndex(news_event_df.Timestamp_ET)
news_event_df.Date = pd.to_datetime(news_event_df.Date)
news_event_df = news_event_df.set_index(['Timestamp_ET'])

cols = ['Ticker', 'Sentiment', 'topic_1', 'topic_2', 'topic_3', 'topic_4']
dates = np.unique(news_event_df.index.get_level_values(0).date) 

for i in tqdm(range(1, len(dates)-1)):
    date = dates[i]
    c = datetime.combine(dates[i-1], time(16,0))
    o = datetime.combine(dates[i],   time(9,30))
    
    not_duplicated = news_event_df.loc[c:o, cols].loc[news_event_df.loc[c:o, 'Ticker'].duplicated(keep=False)==False].reset_index(drop=True)
    duplicated     = news_event_df.loc[c:o, cols].loc[news_event_df.loc[c:o, 'Ticker'].duplicated(keep=False)].groupby('Ticker').mean().reset_index()
    
    not_duplicated['Date'] = dates[i]
    duplicated['Date']     = dates[i]
    not_duplicated['prev_close_date'] = dates[i-1]
    duplicated['prev_close_date']     = dates[i-1]    
    
    if i == 1:
        news_event_df_no_dupl = pd.concat([not_duplicated, duplicated], axis=0)
    else:
        news_event_df_no_dupl = pd.concat([news_event_df_no_dupl, not_duplicated, duplicated], axis=0)
        
        
news_event_df_no_dupl = news_event_df_no_dupl.reset_index(drop=True)
news_event_df_no_dupl.to_csv('./Data/news_event_df_no_dupl.csv', encoding='utf-8')

  0%|          | 0/4520 [00:00<?, ?it/s]

Merge news_event_df with close-to-open and open-to-close returns

In [7]:
news_event_df_no_dupl = load_data('./Data/news_event_df_no_dupl.csv')

df = news_event_df_no_dupl.loc[:, ['prediction_value', 'prev_close_date']]
df = df.rename(columns={'prediction_value':'sentiment'})
df = df.sort_index()
df.to_csv('./Data/news_event_df_no_dupl_minimal.csv', encoding='utf-8')

Generate z_score_event_df file containing all z-scores together with  close-to-open and open-to-close returns

In [18]:
zvals_returns1 = pd.read_csv(extended_model_path+'z_values_127d_mo_to_mc_idiosync_abn_returns_mc_to_mo_2002-2021.csv', encoding='utf-8-sig', index_col=0)   # z_values_returns_z_val_neg_95_
zvals_returns1.index.name = 'Ticker'
zvals_returns1.Date = pd.to_datetime(zvals_returns1.Date)
zvals_returns1 = zvals_returns1.set_index(['Date'], append=True)
zvals_returns1 = zvals_returns1.sort_index()
print(zvals_returns1.shape)

zvals_returns2 = pd.read_csv(extended_model_path+'z_values_127d_mo_to_mc_idiosync_abn_returns_mo_to_mc_2002-2021.csv', encoding='utf-8-sig', index_col=0)   # z_values_returns_z_val_neg_95_
zvals_returns2.index.name = 'Ticker'
zvals_returns2.Date = pd.to_datetime(zvals_returns2.Date)
zvals_returns2 = zvals_returns2.set_index(['Date'], append=True)
zvals_returns2 = zvals_returns2.sort_index()
print(zvals_returns2.shape)

z_score_df = pd.concat([zvals_returns1[['z_value', 'return_t_1']], zvals_returns2['return_t_1']], axis=1)
z_score_df.columns = ['z_score', 'return_mc_to_mo', 'return_mo_to_mc']
z_score_df.to_csv('./Data/z_score_event_df.csv', encoding='utf-8')

(2308915, 15)


Merge news events with z-scores

In [198]:
z_score_df = pd.read_csv('./Data/z_score_event_df.csv', encoding='utf-8')
z_score_df.Date = pd.to_datetime(z_score_df.Date)

news_event_df = pd.read_csv('./Data/news_event_df_no_dupl_minimal.csv', encoding='utf-8')
news_event_df = news_event_df.drop(['prev_close_date'], axis=1)
news_event_df.Date = pd.to_datetime(news_event_df.Date)

alldates = np.unique(z_score_df.Date)
alldates = pd.DataFrame(data={'Date':alldates, 'Date_Index': np.arange(0, len(alldates))})

temp1 = pd.merge(left=z_score_df, right=alldates, on='Date', how='left')
temp1 = temp1.rename(columns={'z_score':'z_score_tp1'})
temp1 = temp1[['Ticker', 'Date_Index', 'z_score_tp1']]
temp1 = temp1.set_index(['Date_Index', 'Ticker'])

temp2 = pd.merge(left=news_event_df, right=alldates, on='Date', how='left')
temp2 = temp2.drop('Date', axis=1)
temp2 = temp2.set_index(['Date_Index', 'Ticker'])

dataset = pd.merge(left=z_score_df, right=alldates, on='Date', how='left')
dataset['Date_Index_Merge'] = dataset.Date_Index+1
dataset = dataset.rename(columns={'return_mc_to_mo':'return_mc_to_mo_tp1', 'return_mo_to_mc':'return_mo_to_mc_tp1'})
dataset = pd.merge(left=dataset, right=temp1, left_on=['Date_Index_Merge', 'Ticker'], right_index=True, how='left')
dataset = pd.merge(left=dataset, right=temp2, left_on=['Date_Index_Merge', 'Ticker'], right_index=True, how='left')
dataset = dataset.drop(['Date_Index', 'Date_Index_Merge'], axis=1)

dataset.to_csv('./Data/z_score_event_df_sentiment.csv', encoding='utf-8')

Daytime News

In [20]:
news_events = pd.read_csv("./Data/news_event_df.csv", encoding='utf-8')
news_events.Timestamp_ET = pd.to_datetime(news_events.Timestamp_ET)
news_events.TradingDate = pd.to_datetime(news_events.TradingDate)
news_events.Date = pd.to_datetime(news_events.Date)

daytime_news = news_events.loc[((news_events.Timestamp_ET.dt.time >= time(9,30)) & (news_events.Timestamp_ET.dt.time <= time(16,0)))==True]
daytime_news = daytime_news.loc[:, ['Ticker', 'Date', 'Sentiment']].reset_index(drop=True)

daytime_news.to_csv('./Data/news_event_df_daytime.csv', encoding='utf-8')